#### Added pipeline_name and tokenizer --(working)

In [1]:
from a360mdk import A360AI

In [2]:
datarepos = a360ai.list_datarepos()
datarepos

,name,description,storage_provider
0,Jupyter R-Python,Test Jupyter R-Python notebook image,aws


In [30]:
a360ai.set_default_datarepo("Jupyter R-Python")

In [28]:
a360ai.list_datasets()

,base_name,extension,size
1,SMSSpamCollection,,483481.0
2,SMSSpamCollection.csv,csv,500350.0
3,df.parquet,parquet,325696.0


In [29]:
df = a360ai.load_dataset("df.parquet")
df

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [4]:
import pandas as pd
X=list(df['message'])[:100]
y=list(df['label'])[:100]
y=list(pd.get_dummies(y,drop_first=True)['spam'])
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [5]:
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(X_train, truncation=True, padding=True,max_length=512)
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=512)

import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    eval_steps=1000,
)

with training_args.strategy.scope():

    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,             # evaluation dataset
    
)

trainer.train()


2022-10-06 05:08:16.962152: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-06 05:08:16.962181: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-06 05:08:18.735948: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-06 05:08:18.735976: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-06 05:08:18.735997: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (notebook-server-danish-huggingface-3052e17-0): /proc/driver/nvidia/version does not exist
2022-10-06 05:08:18.736279: I te

In [6]:
import pandas as pd
X_train = pd.DataFrame(X_train)
X_train.columns =['message']
y_train = pd.DataFrame(y_train)
y_train.columns =['label']
y_train

,label
0,0
1,0
2,0
3,0
4,0
...,...
75,0
76,1
77,0
78,0


In [7]:
my_model = a360ai.get_or_create_model(model_name=f"huggingfacetest2")

/home/jovyan/a360-mdk/a360mdk/controllers/model.py:190: UserWarning: Only the first 15 characters of the model name will be used for deployment and deployment monitoring
  warnings.warn(


In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [9]:
tokenizer

PreTrainedTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [10]:
experiment = my_model.get_or_create_experiment(
    experiment_name = f"huggingfacetest2",
    model_flavor = "huggingface",
    enable_drift_monitoring=False,
    train_features=X_train,
    train_target=y_train,
    feature_names=list(["message"]),
    data_exploration_file="/home/jovyan/01_exploratory-data-analysis.ipynb",
    data_preparation_file="/home/jovyan/02_data-preprocessing.ipynb",
    model_training_file="/home/jovyan/03_model-training.ipynb",
    pipeline_name = "text-classification",
    tokenizer = tokenizer,
    framework = "tf"
    
)


                //-- Experiment Loaded --//
                Model Name: huggingfacetest2
                Experiment Name: huggingfacetest2
                Final Run Id: 537dff7b-6604-4634-88b4-7e78fed0a690
                Model Flavor: huggingface
                Input Signature: message text
                Output Signature: label big_integer
                Data Exploration File: /home/jovyan/01_exploratory-data-analysis.ipynb
                Data Preparation File: /home/jovyan/02_data-preprocessing.ipynb
                Model Training File: /home/jovyan/03_model-huggingface.ipynb
                Drift Monitoring Enabled: False
                Pipe Line: text-classification


In [11]:
test = trainer.evaluate(test_dataset)['eval_loss']
train = trainer.evaluate(train_dataset)['eval_loss']

2022-10-06 05:08:25.850087: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_3"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
input: "Placeholder/_2"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_INT32
      type: DT_INT32
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 20
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024TensorSliceDataset:1"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 83
        }
      }
      shape {
        dim {
          size: 83
        }
      }
      shape {
      }
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
  

In [12]:
test

0.04684942588210106

In [13]:
train

0.08538745641708374

In [14]:
metrics = {'test_score': test, 'train_score': train}
with experiment.run_experiment() as run:
    run.log_metrics(metrics)
    run.log_model(model)

/home/jovyan/a360-mdk/a360mdk/controllers/notebook_save.py:93: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cells = cells.append(final_input_cell, ignore_index=True)


Run - 07f53016-aa00-417a-90a6-d599fa94ec34/trained_model
ONNX opset version set to: 12
Loading pipeline (model: <transformers.models.distilbert.modeling_tf_distilbert.TFDistilBertForSequenceClassification object at 0x7fb2e812bd30>, tokenizer: PreTrainedTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}))
Creating folder Run - 07f53016-aa00-417a-90a6-d599fa94ec34
/!\ Please note TensorFlow doesn't support exporting model > 2Gb /!\
Using framework TensorFlow: 2.9.0, tf2onnx: 1.12.1


/opt/conda/lib/python3.10/site-packages/transformers/convert_graph_to_onnx.py:378: FutureWarning: The `transformers.convert_graph_to_onnx` package is deprecated and will be removed in version 5 of Transformers
  warnings.warn(


Found input input_ids with shape: {0: 'batch', 1: 'sequence'}
Found input attention_mask with shape: {0: 'batch', 1: 'sequence'}
Found output output_0 with shape: {0: 'batch'}


2022-10-06 05:08:34.062873: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_7198"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\021FlatMapDataset:34"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUC

1/1 [==============================] - 1s 1s/step


2022-10-06 05:08:36.042930: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-10-06 05:08:36.043085: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


2022-10-06 05:08:43.085380: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-10-06 05:08:43.085552: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


In [15]:
!pwd

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/jovyan


In [16]:
runs = experiment.list_runs()
runs

,id,dataset,artifact_paths,metric_test_score,metric_train_score,metadata_run_time
0,5957bc14-617a-40bf-979b-0fb1c68fd2e0,fb28f6f9-7256-4038-9267-1fa0c0e22f50,[{'id': 'fc396e96-2f49-468c-93da-3630927f676c'...,0.653457,0.632815,0.000031
1,8512f6e4-31ea-4178-ad68-d16dee7123ae,fb28f6f9-7256-4038-9267-1fa0c0e22f50,[{'id': '1cde3c2a-f1d8-4db6-bf12-e431f2946265'...,0.684225,0.680252,0.000031
2,4d58a949-0b7a-4bbe-9ea6-89b4627881f7,3bd007c7-3e6e-4145-848f-860b423255a2,[{'id': '5f5177ff-5ed0-47ae-b2e8-0fcfada1c691'...,0.657940,0.639467,0.000011
3,ced77ff4-e5c9-4500-8bb9-4b894faa445c,3bd007c7-3e6e-4145-848f-860b423255a2,[{'id': '71a04371-1f86-4229-accd-9db1af3b1efb'...,0.698017,0.694318,0.000012
4,dce86687-afec-487d-96b2-09c75d0d0d62,3bd007c7-3e6e-4145-848f-860b423255a2,[{'id': 'f7b7ff11-5a61-4c8c-a26c-6d95b34518c7'...,0.662687,0.643995,0.000010
5,537dff7b-6604-4634-88b4-7e78fed0a690,3bd007c7-3e6e-4145-848f-860b423255a2,[{'id': '392b25e0-108a-4dbf-b0b3-02ab470682f7'...,0.674387,0.661312,0.000010
6,fe6b2085-ceda-461c-a7e4-7e6705103723,3bd007c7-3e6e-4145-848f-860b423255a2,[{'id': 'a99d8c8a-429c-4cdd-bc1f-59f85febe874'...,0.690580,0.691324,0.000011
7,e01a9b5c-2464-47d6-b2f1-8bb5429dee37,3bd007c7-3e6e-4145-848f-860b423255a2,[{'id': 'd6b95ff5-89d8-4de9-9fae-5b027b601594'...,0.681670,0.678386,0.000011
8,ec0631ea-b6eb-4838-aeb1-df7dd313261e,3bd007c7-3e6e-4145-848f-860b423255a2,[{'id': '2c2e8470-4dcf-489a-a32f-4cfe81ee6a78'...,0.666425,0.650836,0.000011
9,7df18154-f496-4134-8bb9-278825c6e3ad,a164bdf9-f5d3-44d7-baed-f7802bef75fb,[{'id': 'f9efc970-8238-47b1-90e1-6e906620a358'...,0.642181,0.609940,0.000010


In [17]:
runs.sort_values('metric_test_score',ascending=False)

,id,dataset,artifact_paths,metric_test_score,metric_train_score,metadata_run_time
3,ced77ff4-e5c9-4500-8bb9-4b894faa445c,3bd007c7-3e6e-4145-848f-860b423255a2,[{'id': '71a04371-1f86-4229-accd-9db1af3b1efb'...,0.698017,0.694318,0.000012
11,2926be44-c4a4-422f-97c3-cb16162c20c5,899ebaa7-9f92-4ca8-94a5-9da9a2a80cbc,[{'id': 'a3047705-e287-4d12-b84c-96973a94e74e'...,0.695347,0.694966,0.000011
6,fe6b2085-ceda-461c-a7e4-7e6705103723,3bd007c7-3e6e-4145-848f-860b423255a2,[{'id': 'a99d8c8a-429c-4cdd-bc1f-59f85febe874'...,0.690580,0.691324,0.000011
1,8512f6e4-31ea-4178-ad68-d16dee7123ae,fb28f6f9-7256-4038-9267-1fa0c0e22f50,[{'id': '1cde3c2a-f1d8-4db6-bf12-e431f2946265'...,0.684225,0.680252,0.000031
7,e01a9b5c-2464-47d6-b2f1-8bb5429dee37,3bd007c7-3e6e-4145-848f-860b423255a2,[{'id': 'd6b95ff5-89d8-4de9-9fae-5b027b601594'...,0.681670,0.678386,0.000011
5,537dff7b-6604-4634-88b4-7e78fed0a690,3bd007c7-3e6e-4145-848f-860b423255a2,[{'id': '392b25e0-108a-4dbf-b0b3-02ab470682f7'...,0.674387,0.661312,0.000010
8,ec0631ea-b6eb-4838-aeb1-df7dd313261e,3bd007c7-3e6e-4145-848f-860b423255a2,[{'id': '2c2e8470-4dcf-489a-a32f-4cfe81ee6a78'...,0.666425,0.650836,0.000011
10,bdb2d234-0f52-4f9e-bf15-67e5077f725c,899ebaa7-9f92-4ca8-94a5-9da9a2a80cbc,[{'id': '342301fa-61a4-4dac-83fd-a6874fd5d744'...,0.666108,0.646638,0.000010
4,dce86687-afec-487d-96b2-09c75d0d0d62,3bd007c7-3e6e-4145-848f-860b423255a2,[{'id': 'f7b7ff11-5a61-4c8c-a26c-6d95b34518c7'...,0.662687,0.643995,0.000010
2,4d58a949-0b7a-4bbe-9ea6-89b4627881f7,3bd007c7-3e6e-4145-848f-860b423255a2,[{'id': '5f5177ff-5ed0-47ae-b2e8-0fcfada1c691'...,0.657940,0.639467,0.000011


In [18]:
best_run_id = runs.sort_values("metric_test_score", ascending=False).id.values[0]
best_run_id

'ced77ff4-e5c9-4500-8bb9-4b894faa445c'

In [19]:
experiments = my_model.list_experiments()
experiments

,id,bestRun,model_id,experiment_name,best_run_id,model_flavor,input_signature,output_signature,data_exploration_file,data_preparation_file,model_training_file,baseline,train_shape,model_name,updated_at
0,13e53d50-c436-4efb-906e-f375e5558282,"{'id': '537dff7b-6604-4634-88b4-7e78fed0a690',...",95d7fd0b-c18a-4d91-9df6-108035195a4d,huggingfacetest2,537dff7b-6604-4634-88b4-7e78fed0a690,huggingface,message text,label big_integer,/home/jovyan/01_exploratory-data-analysis.ipynb,/home/jovyan/02_data-preprocessing.ipynb,/home/jovyan/03_model-huggingface.ipynb,{},[1],huggingfacetest2,2022-10-06 05:09


In [20]:
my_model.set_final_run(experiment, best_run_id)

In [21]:
# my_model.get_artifact("trained_model",path="/home/jovyan")

In [22]:
# model_from_dict = my_model.get_artifact_dictionary()["trained_model"]

In [23]:
# from numpy import float32
# from onnxruntime import InferenceSession
# from sklearn import pipeline


# class ONNXModel:
#     def __init__(self, path):
#         self.session = InferenceSession(path)
#         self.input_name = self.session.get_inputs()[0].name
#         self.label_name = self.session.get_outputs()[0].name

#     def predict(self, input_data, model_flavor=None):
        
#         if model_flavor ==  "huggingface":
#             return self.session.run([self.label_name], input_data)
#         print("Huggingface model")

#         return self.session.run(
#             [self.label_name], {self.input_name: input_data.astype(float32)}
#         )[0]

#     def transform(self, input_data):
#         return self.session.run(
#             [self.label_name], {self.input_name: input_data.astype(float32)}
#         )[0]

In [24]:
# path = "/home/jovyan/trained_model"
# model = ONNXModel(path)

In [25]:
# input_data = "this is some good string"

In [26]:
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
# input = tokenizer(input_data, return_tensors="tf")
# # ONNX Runtime expects NumPy arrays
# inputs = {k: v.numpy() for k, v in input.items()}


In [27]:
# from scipy.special import softmax
# logits_onnx = model.predict(inputs , "huggingface")
# logits_onnx
# preds = softmax(logits_onnx[0][0])
# print(f"Class: {['ham' if preds.argmax() == 0 else 'spam'][0]}, Probability: {preds.max():.4f}")
